In [1]:
#2013-2016 emergence of populist concerns 
# Frequency of Populist key words 
import pandas as pd
populist_posts = pd.read_csv("all_populist_post_0503.csv")


In [7]:
monthly = populist_posts.groupby(["month_created","year_created"]).agg({"id":"count"})
monthly

,,id
month_created,year_created,
1,2016,1975
2,2016,25479
3,2016,30406
4,2016,22482
5,2016,25183
6,2016,30734
7,2016,28813
8,2016,30206
9,2016,29240


In [ ]:
# Topic Modelling

In [ ]:
# (optional) Word Embedding

In [ ]:
# Sentiment Analysis 